In [1]:
library(ggplot2)

In [2]:
# Function definitions for the inverse-gamma probability distribution.
# The parameters of the distribution (alpha, beta) are as defined here:
# https://en.wikipedia.org/wiki/Inverse-gamma_distribution

pinvgamma <- function(x, alpha, beta) {
    # The cdf of the inverse-gamma distribution.
    return(1 - pgamma(1/x, shape=alpha, rate=beta));
}

qinvgamma <- function(p, alpha, beta) {
    # The quantile function of the inverse-gamma distribution.
    return(1 / qgamma(1 - p, shape=alpha, rate=beta));
}

dinvgamma <- function(x, alpha, beta) {
    # The pdf of the inverse-gamma distribution.
    if (alpha <= 0 | beta <= 0) {
        stop("Shape or scale parameter negative in dinvgamma().\n");
    }
    log_density <- alpha * log(beta) - lgamma(alpha) - (alpha + 1) * log(x) - (beta / x);
    return(exp(log_density));
}

rinvgamma <- function(n, alpha, beta) {
    # Draw n samples from the inverse-gamma distribution.
    return(1 / rgamma(n=n, shape=alpha, rate=beta));
}

# Function definitions for the normal-inverse-gamma distribution.
# The parameters of the distribution (mu, lambda, alpha, beta) are as defined here:
# https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution

dnorminvgamma <- function(x, sigma2, mu, lambda, alpha, beta) {
    # The pdf of the normal-inverse-gamma distribution at x (mean) and sigma2 (variance).
    return(dnorm(x, mu, sqrt(sigma2 / lambda)) * dinvgamma(sigma2, alpha, beta));
}

rnorminvgamma <- function(n, mu, lambda, alpha, beta) {
    # Draw n samples from the normal-inverse-gamma distribution.
    # Returns a matrix where each column contains a (x, sigma2) sample.
    sigma2 = rinvgamma(n, alpha, beta);  # Sample sigma^2 from the inverse-gamma
    x = rnorm(n, mu, sqrt(sigma2 / lambda));  # Sample x from the normal
    return(t(matrix(c(x, sigma2), nrow=n, ncol=2)));
}

In [3]:
# we want the mean of the variance to be 2.75, which is = beta / alpha - 1
# with an SD of about 1 ~= variance of about 1 = beta^2 / (alpha-1)^2(alpha-2)
alpha_0 <- 1
beta_0 <- 0
var <- 10

while (var > 1) {
    # mean <- beta_0 / (alpha_0 - 1)
    alpha_0 <- alpha_0 + 1
    beta_0 <- 2.75 * (alpha_0 - 1)
    var <- (beta_0 ** 2) / ( (alpha_0 - 2) * (alpha_0 - 1) ** 2 )
}
print(c(alpha_0, beta_0, var))


[1] 10.0000000 24.7500000  0.9453125


In [4]:
mu_0 <- 2.3     # The prior mean is centered around 2.3
nu_0 <- 11       # we want SD(x) ~= 0.5 => var(x) ~= 0.25 => nu ~= 0.25 / E[sigma^2] = 0.25 / 2.75 = 11
prior_sample <- rnorminvgamma(100, mu_0, nu_0, alpha_0, beta_0)

In [5]:
means <- prior_sample[seq(1, length(prior_sample), 2)]
variances <- prior_sample[seq(2, length(prior_sample), 2)]
quantile(means, probs = c(0.025, 0.5 - 0.34, 0.5 + 0.34, 0.975))
quantile(variances, probs = c(0.025, 0.975))

2.5%      16%      84%    97.5% 
1.215503 1.725198 2.732280 3.324096

2.5%    97.5% 
1.401017 4.289205

In [6]:
# sample.df <- data.frame(x=seq(-4, 8, length.out=1000))
sample.df <- data.frame(x=-4:8)
ps <- rnorminvgamma(1, mu_0, nu_0, alpha_0, beta_0)
# dnorm(sample.df$x, mean=ps[1], sd=ps[2] ** 0.5)

# prior_plot <- ggplot(sample.df, aes(x=x))

# for (i in seq(1, 5)) {
#     ps <- prior_sample <- rnorminvgamma(1, mu_0, nu_0, alpha_0, beta_0)
#     print(ps)
#     prior_plot <- prior_plot + stat_function(fun=dnorm, args=list(mean=ps[1], sd=ps[2] ** 0.5), color=i)   
# }

# prior_plot + 



In [ ]:
# ggplot(sample.df, aes(x=x)) +
#     stat_function(fun=dnorm, args=list(mean=ps[1], sd=ps[2] ** 0.5)) # , color=1) +   
# #     theme(aspect.ratio=3/4) + 
# #     xlab("X") + 
# #     ylab("P(X)")